In [ ]:
"""
Help me write a blog post no longer than 1,000 words.

The Tool should be able to do 3 things:

-   research a topic on the web
-   extact specific information from documents I provide
-   write a blog post from the research and documents

"""

In [ ]:
"""
Help me dissect academic research papers.

The Tool should be able to do 4 things:

-   extract key phrases, sentences, and paragraphs from the research paper
-   highlight interesting exhibits, tables, and figures from the research paper
-   hihglight the key references and citations from the research paper
-   write a short twitter thread about the research paper using the extracted information

"""